In [1]:
from pymongo import MongoClient
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd
import numpy as np
from math import *
import os
#
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Descriptors

**Complete Databank**

In [2]:
client = MongoClient("localhost:27017")
db_complete = client['COCONUT2020-10']

In [3]:
collection = db_complete.uniqueNaturalProduct
db_complete_only_ring_sugars = pd.DataFrame(list(collection.find({"contains_ring_sugars": True})))

In [6]:
all_tax_df = pd.DataFrame({"taxonomy": db_complete_only_ring_sugars["textTaxa"],
                       "smiles": db_complete_only_ring_sugars["smiles"],
                       "coconut_id": db_complete_only_ring_sugars["coconut_id"],
                       "no_sugar_smiles": db_complete_only_ring_sugars["sugar_free_smiles"]
                      })

In [7]:
#display(all_tax_df)

In [8]:
indexes = []
for i in range(len(all_tax_df.taxonomy)):
    # some entries are empty lists
    if all_tax_df.taxonomy[i] == []:
        pass
    elif all_tax_df.taxonomy[i][0] == "plants":
        indexes.append(i)

In [9]:
plants_tax_df = all_tax_df.loc[indexes[:]]

In [10]:
plants_tax_and_id_df = pd.DataFrame({"taxonomy": plants_tax_df.taxonomy,
                                "coconut_id": plants_tax_df.coconut_id})

In [11]:
#display(plants_tax_and_id_df)

**Sweetcoconut Databank**

In [13]:
client2 = MongoClient("localhost:27017")
db_s = client2["sweetcoconut"]
collection2 = db_s.sweetNaturalProduct
sweetnp = pd.DataFrame(list(collection2.find({"contains_sugar": True})))

In [24]:
sweetnp_with_plants_tax = pd.merge(sweetnp, plants_tax_and_id_df, how="left", on="coconut_id")

In [25]:
display(sweetnp_with_plants_tax)

,_id,coconut_id,smiles,contains_sugar,nosugar_smiles,no_term_sugar_smiles,all_deglycosilated_smiles,sugars_in_np,list_sugars,_class,taxonomy
0,5f5207f9ec7c751be5a0db2f,CNP0220816,O=C1OC2=CC(=CC(OC3OC(CO)C(O)C(O)C3O)=C2C4=C1CC...,True,O=C1OC=2C=C(C=C(O)C2C3=C1CCC3)C,O=C1OC=2C=C(C=C(O)C2C3=C1CCC3)C,[O=C1OC=2C=C(C=C(O)C2C3=C1CCC3)C],{},"[[O1[CH][CH][CH][CH][CH]1, circular, is_termin...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
1,5f5207f9ec7c751be5a0db34,CNP0125332,O=P(O)(O)OCC1OC(OP(=O)(O)O)C(O)C1O,True,O=P(O)(O)OC,O=P(O)(O)OCC1OC(OP(=O)(O)O)C(O)C1O,"[O=P(O)(O)OC, O=P(O)(O)O]",{},"[[O1[CH][CH][CH][CH]1, circular, is_internal]]",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
2,5f5207f9ec7c751be5a0db3a,CNP0151033,O=C(OC1C(OCC2=COC(OC(=O)CC(C)C)C3C2CC(O)C3(O)C...,True,O=C(OCC1(O)C(O)CC2C(=COC(OC(=O)CC(C)C)C21)CO)C,O=C(OC1C(OCC2=COC(OC(=O)CC(C)C)C3C2CC(O)C3(O)C...,"[O=C(O)C=CC1=CC=C(O)C=C1, O=C(OCC1(O)C(O)CC2C(...",{},"[[O1[CH][CH][CH][CH][CH]1, circular, is_intern...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
3,5f5207f9ec7c751be5a0db4a,CNP0298418,O=C([O-])C12CCC(C)(CO)CC2C3=CCC4C(C)(CC5[NH2+]...,True,O=C([O-])C12CCC(C)(CO)CC2C3=CCC4C(C)(CC5[NH2+]...,O=C([O-])C12CCC(C)(CO)CC2C3=CCC4C(C)(CC5[NH2+]...,[O=C([O-])C12CCC(C)(CO)CC2C3=CCC4C(C)(CC5[NH2+...,{},"[[O1[CH][CH][CH][CH]C1, circular, is_terminal]]",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
4,5f5207f9ec7c751be5a0db56,CNP0063031,O=C(OC1CCN2(=O)CC=C(COC(=O)C(O)(C(OC)C)C(O)(C)...,True,O=C(OC1CCN2(=O)CC=C(C)C12)C(=CC)C,O=C(OC1CCN2(=O)CC=C(C)C12)C(=CC)C,[O=C(OC1CCN2(=O)CC=C(C)C12)C(=CC)C],{},"[[[O][CH]C(O)([C]O)C(=O)[O], linear, is_termin...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
49480,5f520880ec7c751be5a75ebd,CNP0391955,O=C(O)C1OC(OC2CCC3(C)C4CC=C5C67C=CCC(C)(CCC6(C...,True,O=C(O)C12CCC3(C(=CCC4C5(C)CCC(O)C(C)(C)C5CCC43...,O=C(O)C12CCC3(C(=CCC4C5(C)CCC(O)C(C)(C)C5CCC43...,[O=C(O)C12CCC3(C(=CCC4C5(C)CCC(O)C(C)(C)C5CCC4...,{},"[[O1[CH][CH][CH][CH][CH]1, circular, is_termin...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
49481,5f520880ec7c751be5a75ec5,CNP0345297,O=C1C(OCC2OC(O)C(O)C(O)C2O)=C(OC=3C=C(O)C=C(O)...,True,O=C1C(OC)=C(OC=2C=C(O)C=C(O)C12)C=3C=CC(O)=CC3,O=C1C(OC)=C(OC=2C=C(O)C=C(O)C12)C=3C=CC(O)=CC3,[O=C1C(OC)=C(OC=2C=C(O)C=C(O)C12)C=3C=CC(O)=CC3],{},"[[O1[CH][CH][CH][CH][CH]1, circular, is_termin...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
49482,5f520880ec7c751be5a75ec6,CNP0159133,O[C]1[CH][C](O)[C]2[CH][C](OC3OC(CO)C(O)C(O)C3...,True,OC1CC(O)C2CC(O)C([OH+]C2C1)C3CC(OC)C(O)C(OC)C3,OC1CC(O)C2CC(O)C([OH+]C2C1)C3CC(OC)C(O)C(OC)C3,[OC1CC(O)C2CC(O)C([OH+]C2C1)C3CC(OC)C(O)C(OC)C3],{},"[[O1[CH][CH][CH][CH][CH]1, circular, is_termin...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
49483,5f520880ec7c751be5a75ed0,CNP0298814,O=C(O)C12C3=C(CCC2(C)C(CC1)C(C)CCC(=C)C(C)(C)C...,True,O=C(O)C12C3=C(CCC2(C)C(CC1)C(C)CCC(=C)C(C)(C)C...,O=C(O)C12C3=C(CCC2(C)C(CC1)C(C)CCC(=C)C(C)(C)C...,[O=C(O)C12C3=C(CCC2(C)C(CC1)C(C)CCC(=C)C(C)(C)...,{},"[[O1[CH][CH][CH][CH]C1, circular, is_terminal]...",de.unijena.cheminf.sweetcoconut.mongomodel.Swe...,NaN
